# Imports

In [ ]:
import pandas as pd # dataframes
from sklearn.preprocessing import RobustScaler # scale to zero mean and unit variance
from sklearn.decomposition import PCA # primary component analysis
import matplotlib.pyplot as plt # plots
import seaborn as sns # advanced plots
import numpy as np # ndarrays

# Preprocess the Data Set

# Impute Missing Values
- Estimate the accuracy of the imputation for each feature

# Principal Component Analysis
- Transform the data into the principal components from phase one
- Would it make sense to retrain the PCA for the new data?

# Perform a Cluster Analysis
-Compare and discuss the results of the Anomaly Detection & Cluster Analysis

# Visualize Results so far
- Plot datapoints of the first PCs
- Decision Boundary of the Anomaly Detector
    - Color code clusters
    - Separate marker style for imputed data points